In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
import openslide
import pandas as pd
import pickle

In [4]:
from fastai.callbacks.csv_logger import CSVLogger

In [5]:
from object_detection_fastai.helper.object_detection_helper import *
from object_detection_fastai.helper.wsi_loader import *
from object_detection_fastai.loss.RetinaNetFocalLoss import RetinaNetFocalLoss
from object_detection_fastai.models.RetinaNet import RetinaNet
from object_detection_fastai.callbacks.callbacks import BBLossMetrics, BBMetrics, PascalVOCMetric, PascalVOCMetricByDistance

In [6]:
slides_train = ["15_EIPH_568320 berliner blau.svs", 
                         "22_EIPH_575216 Berliner Blau.svs", 
                         "30_EIPH_588355 Berliner Blau.svs", 
                         "19_EIPH_566933 L Tunrbull blue.svs", 
                         "02_EIPH_574162 Turnbull blue-001.svs", 
                        ]

slides_val = ["27_EIPH_571557 Turnbull blue.svs", "17_EIPH_575796 Berliner Blau.svs"]

In [7]:
grade_list = [0, 1]
annotations_path = Path("../../Statistics/SDATA_Final_Annotations.pkl")
annotations = pd.read_pickle(annotations_path)
annotations["image_name"] = [name.replace("tiff","svs") for name in annotations["image_name"]]
annotations = annotations[annotations["grade"].isin(grade_list)]
annotations_train = annotations[annotations["image_name"].isin(slides_train)]
annotations_val = annotations[annotations["image_name"].isin(slides_val)]
annotations_train.head()

,id,image_id,image_set,species,image_name,image_type,grade,vector,unique_identifier,user_id,deleted,last_editor,data_set_name,version
19314,1453268,3549,246,equine,15_EIPH_568320 berliner blau.svs,BerlinerBlau,0,"{'x1': 22287, 'x2': 22389, 'y1': 25395, 'y2': ...",190231f1-39c5-4de5-910a-c137ffc38e76,1,True,1,SDATA,Cluster
19315,1453269,3549,246,equine,15_EIPH_568320 berliner blau.svs,BerlinerBlau,0,"{'x1': 23030, 'x2': 23141, 'y1': 25124, 'y2': ...",74b8d1b6-4632-4c8c-b304-90c12685f017,1,False,1,SDATA,Inference
19316,1453270,3549,246,equine,15_EIPH_568320 berliner blau.svs,BerlinerBlau,0,"{'x1': 19613, 'x2': 19683, 'y1': 5604, 'y2': 5...",7862ba0e-99f4-407a-b397-625870f3de28,1,True,1,SDATA,Cluster
19317,1453271,3549,246,equine,15_EIPH_568320 berliner blau.svs,BerlinerBlau,0,"{'x1': 4342, 'x2': 4434, 'y1': 13810, 'y2': 13...",3386d235-9c38-406c-9b06-cd44031513e0,1,False,1,SDATA,Inference
19318,1453272,3549,246,equine,15_EIPH_568320 berliner blau.svs,BerlinerBlau,0,"{'x1': 10015, 'x2': 10089, 'y1': 9697, 'y2': 9...",727b28a3-2e05-42e7-b62b-2e065503af4d,1,True,1,SDATA,Cluster


In [8]:
slides_path = Path("../../../Slides")
files = {slide.name: slide for slide in slides_path.rglob("*.svs")  if slide.name in slides_train + slides_val}
files

{'02_EIPH_574162 Turnbull blue-001.svs': PosixPath('../../../Slides/Equine/02_EIPH_574162 Turnbull blue-001.svs'),
 '15_EIPH_568320 berliner blau.svs': PosixPath('../../../Slides/Equine/15_EIPH_568320 berliner blau.svs'),
 '17_EIPH_575796 Berliner Blau.svs': PosixPath('../../../Slides/Equine/17_EIPH_575796 Berliner Blau.svs'),
 '19_EIPH_566933 L Tunrbull blue.svs': PosixPath('../../../Slides/Equine/19_EIPH_566933 L Tunrbull blue.svs'),
 '22_EIPH_575216 Berliner Blau.svs': PosixPath('../../../Slides/Equine/22_EIPH_575216 Berliner Blau.svs'),
 '27_EIPH_571557 Turnbull blue.svs': PosixPath('../../../Slides/Equine/27_EIPH_571557 Turnbull blue.svs'),
 '30_EIPH_588355 Berliner Blau.svs': PosixPath('../../../Slides/Equine/30_EIPH_588355 Berliner Blau.svs')}

In [9]:
tfms = get_transforms(do_flip=True,
                      flip_vert=True,
                      #max_rotate=90,
                      max_lighting=0.0,
                      max_zoom=1.,
                      max_warp=0.0,
                      p_affine=0.5,
                      p_lighting=0.0,
                      #xtra_tfms=xtra_tfms,
                     )
tfms

([RandTransform(tfm=TfmCrop (crop_pad), kwargs={'row_pct': (0, 1), 'col_pct': (0, 1), 'padding_mode': 'reflection'}, p=1.0, resolved={}, do_run=True, is_random=True, use_on_y=True),
  RandTransform(tfm=TfmAffine (dihedral_affine), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True, use_on_y=True),
  RandTransform(tfm=TfmAffine (rotate), kwargs={'degrees': (-10.0, 10.0)}, p=0.5, resolved={}, do_run=True, is_random=True, use_on_y=True)],
 [RandTransform(tfm=TfmCrop (crop_pad), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True, use_on_y=True)])

In [10]:
size = 1024 
level = 0
bs = 16
train_images = 2500
val_images = 1500

In [11]:
def get_y_func(x):
    return x.y

In [12]:
anchors = create_anchors(sizes=[(32,32)], ratios=[1], scales=[0.6, 0.7, 0.9, 1.25, 1.5])

In [13]:
for i in range(len(slides_train)):
    
    torch.cuda.empty_cache()

    train_files = []
    val_files = []
    
    experiment_name = "EquineVsEquine-Ablation_{}".format(i)

    for image_name in slides_train[:i+1]:

        annotations = annotations_train[annotations_train["image_name"] == image_name]
        annotations = annotations[annotations["deleted"] == False]

        slide_path = files[image_name]
        labels =  list(annotations["grade"])
        bboxes = [[vector["x1"], vector["y1"], vector["x2"], vector["y2"]] for vector in annotations["vector"]]
        
        train_files.append(SlideContainer(slide_path, y=[bboxes, labels],  level=level, width=size, height=size))

    for image_name in annotations_val["image_name"].unique():

        annotations = annotations_val[annotations_val["image_name"] == image_name]
        annotations = annotations[annotations["deleted"] == False]

        slide_path = files[image_name]
        labels =  list(annotations["grade"])
        bboxes = [[vector["x1"], vector["y1"], vector["x2"], vector["y2"]] for vector in annotations["vector"]]
        
        val_files.append(SlideContainer(slide_path, y=[bboxes, labels],  level=level, width=size, height=size))
    
    train_files = list(np.random.choice(train_files, train_images))
    valid_files = list(np.random.choice(val_files, val_images))
    
    train =  ObjectItemListSlide(train_files, path=slides_path)
    valid = ObjectItemListSlide(valid_files, path=slides_path)
    item_list = ItemLists(slides_path, train, valid)
    lls = item_list.label_from_func(get_y_func, label_cls=SlideObjectCategoryList) #
    lls = lls.transform(tfms, tfm_y=True, size=size)
    data = lls.databunch(bs=bs, collate_fn=bb_pad_collate).normalize()
    
    crit = RetinaNetFocalLoss(anchors)
    encoder = create_body(models.resnet18, True, -2)
    model = RetinaNet(encoder, n_classes=data.train_ds.c, n_anchors=5, sizes=[32], chs=128, final_bias=-4., n_conv=3)
    
    voc = PascalVOCMetricByDistance(anchors, size, [str(i) for i in data.train_ds.y.classes[1:]], radius=40)
    learn = Learner(data, model, loss_func=crit, callback_fns=[BBMetrics, partial(CSVLogger, append=False, filename=experiment_name)], #BBMetrics, ShowGraph
                    metrics=[voc])

    learn.split([model.encoder[6], model.c5top5])
    learn.freeze_to(-2)
    
    learn.unfreeze()
    learn.fit_one_cycle(10, 1e-3)
        
    learn.destroy() 

epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,time
0,0.469579,0.345661,0.701265,0.079732,0.265928,0.709335,0.693195,04:33
1,0.284749,0.350032,0.733146,0.088310,0.261722,0.593066,0.873226,04:32
2,0.245854,0.265192,0.758319,0.056456,0.208735,0.792585,0.724053,04:35
3,0.211866,0.237866,0.767694,0.047810,0.190056,0.802351,0.733038,04:36
4,0.193424,0.202834,0.808689,0.040668,0.162166,0.824270,0.793108,04:35
5,0.176332,0.190721,0.811456,0.039386,0.151334,0.835243,0.787670,04:33
6,0.147322,0.167196,0.837570,0.037650,0.129546,0.822587,0.852553,04:30
7,0.128734,0.173157,0.833292,0.034309,0.138848,0.833602,0.832981,04:31
8,0.111917,0.163899,0.847159,0.033180,0.130719,0.830110,0.864207,06:32
9,0.106396,0.170649,0.836666,0.033368,0.137281,0.830621,0.842712,05:57


this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,time
0,0.524471,0.364557,0.728146,0.076045,0.288512,0.766545,0.689747,04:59
1,0.313123,0.428950,0.637779,0.075197,0.353752,0.721300,0.554258,04:32
2,0.287918,0.249710,0.786437,0.058193,0.191516,0.763338,0.809537,04:37
3,0.227141,0.214716,0.796481,0.056292,0.158424,0.716145,0.876817,04:58
4,0.219545,0.194344,0.800561,0.043892,0.150452,0.703398,0.897724,04:41
5,0.214433,0.174808,0.845433,0.043090,0.131718,0.780972,0.909893,04:37
6,0.175488,0.182610,0.845470,0.042063,0.140547,0.844966,0.845974,04:41
7,0.155544,0.183726,0.861273,0.059628,0.124098,0.839744,0.882803,04:36
8,0.145481,0.163836,0.856220,0.031656,0.132180,0.840275,0.872165,04:37
9,0.132507,0.159509,0.860105,0.034196,0.125312,0.842343,0.877868,04:40


this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,time
0,0.460342,0.335582,0.716597,0.094106,0.241476,0.748784,0.684411,04:37
1,0.281561,0.311548,0.745376,0.078744,0.232804,0.618706,0.872046,04:36
2,0.257101,0.254079,0.758302,0.064582,0.189497,0.654222,0.862382,04:43
3,0.244176,0.256468,0.822567,0.063702,0.192767,0.832342,0.812792,04:43
4,0.213527,0.194834,0.823556,0.047877,0.146957,0.788819,0.858293,08:02
5,0.185173,0.202025,0.818472,0.049196,0.152829,0.760845,0.876099,04:39
6,0.167739,0.173468,0.849685,0.041510,0.131958,0.860088,0.839283,04:35
7,0.152691,0.153520,0.865172,0.039290,0.114230,0.837939,0.892405,04:54
8,0.138498,0.148442,0.874517,0.033393,0.115050,0.846498,0.902537,05:00
9,0.125679,0.148383,0.870212,0.032307,0.116077,0.842263,0.898161,04:59


this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,time
0,0.442771,0.329431,0.790323,0.090429,0.239002,0.757940,0.822706,05:19
1,0.289443,0.439276,0.782840,0.100728,0.338549,0.799568,0.766112,04:47
2,0.241011,0.235436,0.807770,0.068925,0.166511,0.752731,0.862810,04:37
3,0.238234,0.214808,0.818142,0.051655,0.163152,0.753422,0.882863,04:30
4,0.203005,0.203834,0.793307,0.049292,0.154542,0.810238,0.776377,04:38
5,0.191935,0.175441,0.832657,0.039569,0.135872,0.802445,0.862869,04:40
6,0.162965,0.157123,0.851900,0.037401,0.119722,0.830373,0.873427,04:36
7,0.152298,0.148198,0.856383,0.032117,0.116081,0.808596,0.904171,04:33
8,0.135099,0.146903,0.868964,0.030483,0.116420,0.830912,0.907016,04:36
9,0.132667,0.136382,0.876774,0.030663,0.105720,0.858026,0.895523,04:35


this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,time
0,0.478968,0.275429,0.773063,0.078657,0.196772,0.729741,0.816384,04:34
1,0.285793,0.212532,0.823302,0.061201,0.151331,0.787845,0.858759,04:34
2,0.290780,0.250500,0.785081,0.055205,0.195295,0.722746,0.847416,04:39
3,0.264420,0.229522,0.791837,0.060430,0.169092,0.706084,0.877590,04:39
4,0.219859,0.223348,0.767272,0.053423,0.169924,0.658985,0.875558,07:49
5,0.205387,0.198309,0.783519,0.046804,0.151505,0.804211,0.762826,04:57
6,0.177343,0.182771,0.813357,0.036589,0.146182,0.723796,0.902918,04:32
7,0.164046,0.158586,0.858905,0.035076,0.123510,0.821487,0.896322,04:36
8,0.148435,0.135500,0.875000,0.030182,0.105318,0.854158,0.895843,04:34
9,0.140045,0.136487,0.877828,0.031770,0.104717,0.855515,0.900140,04:53


this Learner object self-destroyed - it still exists, but no longer usable
